In [2]:
from op_analytics.datapipeline.chains.load import load_chain_metadata
df = load_chain_metadata()


2025-05-23 18:00:50 [info     ] loaded vault from .env file    filename=vault.py lineno=32 process=61751
2025-05-23 18:00:50 [debug    ] loaded vault: 28 items         filename=vault.py lineno=79 process=61751
2025-05-23 18:00:50 [debug    ] Found vault variable GOOGLE_SERVICE_ACCOUNT (has JSON key) filename=gcpauth.py lineno=18 process=61751
2025-05-23 18:00:51 [info     ] Loaded OP chains metadata from /Users/chuxinhuang/work/op-analytics/op_chains_tracking/inputs/chain_metadata_raw.csv filename=load.py lineno=85 process=61751
2025-05-23 18:00:51 [info     ] [REPO vs. GSHEETS] ERROR: Chain Metadata is different filename=load.py lineno=97 process=61751


In [3]:
import polars as pl

df.filter(pl.col("oplabs_db_schema").is_not_null() & pl.col("system_config_proxy").is_not_null()).select(
    pl.col("oplabs_db_schema").alias("chain"),
    pl.col("mainnet_chain_id").alias("chain_id"),
    pl.col("rpc_url"),
    pl.col("system_config_proxy")
)

chain,chain_id,rpc_url,system_config_proxy
str,i64,str,str
"""zora""",7777777,"""https://rpc.zora.energy/""","""0xA3cAB0126d5F504B071b81a3e8A2…"
"""base""",8453,"""https://mainnet.base.org/""","""0x73a79Fab69143498Ed3712e519A8…"
"""mode""",34443,"""https://mainnet.mode.network""","""0x5e6432F18Bc5d497B1Ab2288a025…"
"""lisk""",1135,"""https://rpc.api.lisk.com""","""0x05f23282FFDCA8286E4738C1aF79…"
"""metal""",1750,"""https://rpc.metall2.com/""","""0x7BD909970B0EEdcF078De6Aeff23…"
…,…,…,…
"""ethernity""",183,"""https://mainnet.ethernitychain…","""0x20c3035C92bdB4C461242571EeAc…"
"""superseed""",5330,"""https://mainnet.superseed.xyz""","""0x525a2744134805516a45B8abb6Aa…"
"""unichain""",130,"""https://mainnet.unichain.org""","""0xc407398d063f942feBbcC6F80a15…"
